In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import LRScheduler
from torch.nn import Transformer
import json
import random
import copy
import warnings
import math
import os

In [2]:
class CharTransformer(nn.Module):
    def __init__(self, vocab_size, d_model=256, nhead=4, num_layers=4, d_ff=1024, max_len=32, 
                 dropout=0.3, device="cuda", pad_token_id=0, start_token_id=1, end_token_id=2):
        super(CharTransformer, self).__init__()
        self.device = device

        self.pad_token_id = pad_token_id
        self.start_token_id = start_token_id
        self.end_token_id = end_token_id
        
        self.embedding = nn.Embedding(vocab_size, d_model).to(device)
        self.dropout = nn.Dropout(dropout)  # Dropout after embedding
        
        # Create sinusoidal positional encodings
        self.register_buffer("positional_encoding", self._generate_sinusoidal_encoding(max_len, d_model))
        
        self.transformer = Transformer(
            d_model=d_model, nhead=nhead, num_encoder_layers=num_layers, 
            num_decoder_layers=num_layers, dim_feedforward=d_ff, activation="gelu", batch_first=True, 
            dropout=dropout 
        ).to(device)

        self.fc_out = nn.Linear(d_model, vocab_size).to(device)
    
    def _generate_sinusoidal_encoding(self, max_len, d_model):
        position = torch.arange(max_len, device=self.device).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2, device=self.device) * -(math.log(10000.0) / d_model))
        encoding = torch.zeros(max_len, d_model, device=self.device)
        encoding[:, 0::2] = torch.sin(position * div_term)
        encoding[:, 1::2] = torch.cos(position * div_term)
        return encoding

    def _generate_square_subsequent_mask(self, sz):
        mask = torch.tril(torch.ones(sz, sz))
        return torch.log(mask).to(self.device)
        
    def forward(self, src, tgt, feature_mask, tgt_is_causal=False, src_mask=None, tgt_mask=None, 
                src_key_padding_mask=None, tgt_key_padding_mask=None):
        src, tgt = src.to(self.device), tgt.to(self.device)
        
        # Compute embeddings and apply dropout
        src_emb = self.embedding(src)
        tgt_emb = self.embedding(tgt)

        # Apply positional encoding
        src_emb += (1 - feature_mask[:, :src.shape[1], None]) * self.positional_encoding[:src.shape[1], :]
        tgt_emb += self.positional_encoding[:tgt.shape[1], :].unsqueeze(0)
        
        src_emb = self.dropout(src_emb)
        tgt_emb = self.dropout(tgt_emb)

        # Compute key padding masks if not provided
        if src_key_padding_mask is None:
            src_key_padding_mask = (src == self.pad_token_id) 
        if tgt_key_padding_mask is None:
            tgt_key_padding_mask = (tgt == self.pad_token_id)
        if tgt_is_causal: 
            if tgt_mask is None: 
                tgt_mask = self._generate_square_subsequent_mask(tgt.shape[1])
            else:
                tgt_mask += self._generate_square_subsequent_mask(tgt.shape[1])

        # Transformer forward pass
        transformer_output = self.transformer(
            src_emb, tgt_emb, 
            src_mask=src_mask, 
            tgt_mask=tgt_mask,
            src_key_padding_mask=src_key_padding_mask,
            tgt_key_padding_mask=tgt_key_padding_mask,
            tgt_is_causal=tgt_is_causal
        )
        output = self.fc_out(transformer_output)
        return output
    
    def generate(self, src, feature_mask, max_len=32, beam_size=5):
        self.eval()
        src, feature_mask = src.to(self.device), feature_mask.to(self.device)

        # Initialize beams: (sequence, log probability)
        beams = torch.full((1, 1), self.start_token_id, device=self.device)  
        beam_scores = torch.zeros(1, device=self.device)  # Log probabilities

        completed_sequences = []  # Store completed sequences

        for _ in range(max_len):
            # Expand `src` to match the number of beams
            src_expanded = src.expand(beams.shape[0], -1)
            feature_mask_expanded = feature_mask.expand(beams.shape[0], -1)

            # Forward pass on all beams at once
            out = self.forward(src_expanded, beams, feature_mask_expanded)  
            logits = out[:, -1, :]  # Get last-step logits (shape: [beams, vocab_size])
            log_probs = torch.log_softmax(logits, dim=-1)  # Convert logits to log-probabilities

            # Get top-k candidates for each beam (shape: [beams, beam_size])
            topk_log_probs, topk_ids = log_probs.topk(beam_size, dim=-1)

            # Compute new scores by adding log probabilities (broadcasted)
            expanded_scores = beam_scores.unsqueeze(1) + topk_log_probs  # Shape: [beams, beam_size]
            expanded_scores = expanded_scores.view(-1)  # Flatten to [beams * beam_size]

            # Get top-k overall candidates
            topk_scores, topk_indices = expanded_scores.topk(beam_size)

            # Convert flat indices to beam/token indices
            beam_indices = topk_indices // beam_size  # Which original beam did this come from?
            token_indices = topk_indices % beam_size  # Which token was selected?

            # Append new tokens to sequences
            new_beams = torch.cat([beams[beam_indices], topk_ids.view(-1, 1)[topk_indices]], dim=-1)

            # Check for completed sequences
            eos_mask = (new_beams[:, -1] == self.end_token_id)
            if eos_mask.any():
                for i in range(beam_size):
                    if eos_mask[i]:
                        completed_sequences.append((new_beams[i], topk_scores[i]))

            # Keep only unfinished sequences
            beams = new_beams[~eos_mask]
            beam_scores = topk_scores[~eos_mask]

            # If all sequences finished, stop early
            if len(beams) == 0 or len(completed_sequences) >= beam_size:
                break

        # Choose the best sequence from completed ones
        if completed_sequences:
            best_sequence = max(completed_sequences, key=lambda x: x[1])[0]
        else:
            best_sequence = beams[0]  # If no sequence completed, return best unfinished one

        return best_sequence
    
    def greedy_batch_decode(self, src, feature_mask, max_len=32):
        self.eval()
        batch_size = src.shape[0]
        src, feature_mask = src.to(self.device), feature_mask.to(self.device)
        outputs = torch.full((batch_size, 1), self.start_token_id, device=self.device)
        ended = torch.zeros(batch_size, dtype=torch.bool, device=self.device)
        
        for _ in range(max_len):
            out = self.forward(src, outputs, feature_mask)
            next_tokens = out[:, -1, :].argmax(dim=-1, keepdim=True)
            outputs = torch.cat([outputs, next_tokens], dim=1)
            ended |= (next_tokens.squeeze() == self.end_token_id)
            if ended.all():
                break
        
        return outputs

In [3]:
# Data preparation

def prepare_data(data, val_size=1000, train_size=2000, test_ratio=0.4):
    paradigms = list(data.items())
    random.shuffle(paradigms)
    
    val_set = paradigms[:val_size]
    train_test_set = paradigms[val_size:val_size + train_size]
    
    train_size = int((1 - test_ratio) * len(train_test_set))
    train_set = train_test_set[:train_size]
    test_set = train_test_set[train_size:]
    
    return train_set, test_set, val_set

def generate_examples(paradigm):
    lemma = list(paradigm[0])  # Convert lemma to list of characters
    forms = paradigm[1]
    examples = []
    
    for tag, form in forms.items():
        src = ['<s>', f'<{tag}>'] + lemma + ['</s>']
        tgt = ['<s>'] + list(form) + ['</s>']
        examples.append((src, tgt))
    
    return examples

# Load dataset
with open("/home/minhk/Assignments/CSCI 5801/project/data/processed/eng_v.json", "r") as f:
    data = json.load(f)

train_set, test_set, val_set = prepare_data(data, val_size=1000, train_size=1000, test_ratio=0.1)

train_examples = [ex for paradigm in train_set for ex in generate_examples(paradigm)]
test_examples = [ex for paradigm in test_set for ex in generate_examples(paradigm)]
val_examples = [ex for paradigm in val_set for ex in generate_examples(paradigm)]

print("Train examples:", train_examples[:5])  # Show first 5 examples
print("Test examples:", test_examples[:5])
print("Validation examples:", val_examples[:5])

Train examples: [(['<s>', '<PRS>', 'r', 'a', 'f', 't', '</s>'], ['<s>', 'r', 'a', 'f', 't', '</s>']), (['<s>', '<3SG>', 'r', 'a', 'f', 't', '</s>'], ['<s>', 'r', 'a', 'f', 't', 's', '</s>']), (['<s>', '<PST>', 'r', 'a', 'f', 't', '</s>'], ['<s>', 'r', 'a', 'f', 't', 'e', 'd', '</s>']), (['<s>', '<PRS.PTCP>', 'r', 'a', 'f', 't', '</s>'], ['<s>', 'r', 'a', 'f', 't', 'i', 'n', 'g', '</s>']), (['<s>', '<PST.PTCP>', 'r', 'a', 'f', 't', '</s>'], ['<s>', 'r', 'a', 'f', 't', 'e', 'd', '</s>'])]
Test examples: [(['<s>', '<PRS>', 'c', 'r', 'i', 'n', 'c', 'h', '</s>'], ['<s>', 'c', 'r', 'i', 'n', 'c', 'h', '</s>']), (['<s>', '<3SG>', 'c', 'r', 'i', 'n', 'c', 'h', '</s>'], ['<s>', 'c', 'r', 'i', 'n', 'c', 'h', 'e', 's', '</s>']), (['<s>', '<PST>', 'c', 'r', 'i', 'n', 'c', 'h', '</s>'], ['<s>', 'c', 'r', 'i', 'n', 'c', 'h', 'e', 'd', '</s>']), (['<s>', '<PRS.PTCP>', 'c', 'r', 'i', 'n', 'c', 'h', '</s>'], ['<s>', 'c', 'r', 'i', 'n', 'c', 'h', 'i', 'n', 'g', '</s>']), (['<s>', '<PST.PTCP>', 'c', 'r',

In [4]:
class InverseSquareLRWithWarmup(LRScheduler):
    """
    Implements an inverse square learning rate scheduler with warmup steps.
    
    During warmup, the learning rate increases linearly from init_lr to max_lr.
    After warmup, the learning rate decreases according to the inverse square of the step number:
    lr = max_lr * (warmup_steps / step)^2 for step > warmup_steps
    
    Args:
        optimizer (Optimizer): Wrapped optimizer.
        init_lr (float): Initial learning rate during warmup phase. Default: 0.0
        max_lr (float): Maximum learning rate after warmup phase. Default: 0.1
        warmup_steps (int): Number of warmup steps. Default: 1000
        last_epoch (int): The index of the last epoch. Default: -1
    """
    
    def __init__(self, optimizer, init_lr=0.0, max_lr=0.001, warmup_steps=1000, last_epoch=-1):
        self.init_lr = init_lr
        self.max_lr = max_lr
        self.warmup_steps = warmup_steps
        super(InverseSquareLRWithWarmup, self).__init__(optimizer, last_epoch)
        
    def get_lr(self):
        if not self._get_lr_called_within_step:
            warnings.warn("To get the last learning rate computed by the scheduler, "
                          "please use `get_last_lr()`.")
        
        if self.last_epoch < self.warmup_steps:
            # Linear warmup phase
            alpha = self.last_epoch / self.warmup_steps
            return [self.init_lr + alpha * (self.max_lr - self.init_lr) for _ in self.base_lrs]
        else:
            # Inverse square decay phase
            decay_factor = math.sqrt(self.warmup_steps / self.last_epoch)
            return [self.max_lr * decay_factor for _ in self.base_lrs]
            
    def _get_closed_form_lr(self):
        if self.last_epoch < self.warmup_steps:
            # Linear warmup phase
            alpha = self.last_epoch / self.warmup_steps
            return [self.init_lr + alpha * (self.max_lr - self.init_lr) for _ in self.base_lrs]
        else:
            # Inverse square decay phase
            decay_factor = (self.warmup_steps / self.last_epoch) ** 2
            return [self.max_lr * decay_factor for _ in self.base_lrs]

In [5]:
# Sampler for scheduled learning, gradually replaces ground truth (teacher forcing) with model input

class ScheduledSampler():
    def __init__(self, base_rate=0.5, warmup_steps=1000):
        self.base_rate = base_rate
        self.warmup_steps = warmup_steps
        self.step_count = 0        
        self.sampling_rate = 1
    
    def step(self):
        self.step_count += 1
        if self.step_count > self.warmup_steps:
            self.sampling_rate = self.base_rate + (1 - self.base_rate) * math.sqrt(self.warmup_steps / self.step_count)

    def sample(self, logits, truth_ids):
        """
        Selects truth_ids with probability `sampling_rate`, 
        otherwise samples using Gumbel noise.
        """
        batch_size, seq_len, vocab_size = logits.shape
        
        # Decide per-token whether to take ground truth (1) or Gumbel sample (0)
        mask = torch.bernoulli(torch.full((batch_size, seq_len), self.sampling_rate, device=logits.device, dtype=float)).bool()
        
        # Gumbel-sampled predictions
        gumbel_preds = self._gumbel_sample(logits)
        
        # Use ground truth where mask == True, else use gumbel_preds
        return torch.where(mask, truth_ids, gumbel_preds)
    
    def _gumbel_sample(self, logits):
        gumbel_noise = -torch.log(-torch.log(torch.rand_like(logits)))  # Generate Gumbel noise
        return (logits + gumbel_noise).argmax(dim=-1)  # Apply Gumbel noise and take the argmax

In [6]:
# Tokenizer
def tokenize(sequence, char_to_idx):
    return [char_to_idx[char] for char in sequence]

# Build character vocabulary
all_chars = set()
for word, inflect in data.items():
    all_chars.update(word)
    for tag, forms in inflect.items():
        all_chars.add(f"<{tag}>")
        all_chars.update(forms)
print(all_chars)
char_to_idx = {char: i for i, char in enumerate(sorted(all_chars), start=3)}  # Reserve 0, 1, 2 for special tokens
char_to_idx['<pad>'] = 0
char_to_idx['<s>'] = 1
char_to_idx['</s>'] = 2
idx_to_char = {
    i: char for char, i in char_to_idx.items()
}
vocab_size = len(char_to_idx)
max_len = 32

device = "cuda" if torch.cuda.is_available() else "cpu"
model = CharTransformer(vocab_size, device=device, max_len=max_len)

def pad_sequence(sequence, max_len, pad_token='<pad>'):
    return sequence + [pad_token] * (max_len - len(sequence))

def create_feature_mask(sequence):
    """Create a feature mask where tags (enclosed in < >) are marked as 1, else 0."""
    return torch.tensor([1 if char.startswith('<') and char.endswith('>') else 0 for char in sequence], device=device)

def create_padding_mask(sequence, pad_token='<pad>'):
    """Create a padding mask where padding tokens are marked as True (to be ignored)."""
    return (sequence == pad_token)

def train_model(model, train_examples, test_examples, epochs=1000, batch_size=256, patience=20):
    optimizer = optim.AdamW(model.parameters(), betas=(0.99, 0.98))
    scheduler = InverseSquareLRWithWarmup(optimizer, init_lr=1e-5, max_lr=1e-3, warmup_steps=4000)
    criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
    # sampler = ScheduledSampler(base_rate=0.5, warmup_steps=4000)

    pad_token = char_to_idx['<pad>']
    best_test_loss = float('inf')  # Initialize the best test loss to a very large value
    best_model_state = copy.deepcopy(model.state_dict())  # Store best model parameters
    
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        random.shuffle(train_examples)
        
        for i in range(0, len(train_examples), batch_size):
            batch = train_examples[i:i+batch_size]
            src_batch, tgt_batch = zip(*batch)

            max_batch_len = max(max(len(s) for s in src_batch), max(len(t) for t in tgt_batch))
            
            # Pad sequences to the maximum length (max_len) in the batch
            src_padded = [pad_sequence(seq, max_batch_len) for seq in src_batch]
            tgt_padded = [pad_sequence(seq, max_batch_len) for seq in tgt_batch]
            
            # Convert padded sequences to tensors
            src_tensor = torch.tensor([tokenize(seq, char_to_idx) for seq in src_padded], device=device)
            tgt_tensor = torch.tensor([tokenize(seq, char_to_idx) for seq in tgt_padded], device=device)
            
            # Create the feature mask
            feature_mask_src = torch.stack([create_feature_mask(seq) for seq in src_padded], dim=0)

            # Shift target tensor for teacher forcing (the model will predict next token)
            tgt_input = tgt_tensor[:, :-1]  # Remove the last token (it's not used as input)
            tgt_expected = tgt_tensor[:, 1:]  # The target sequence for the loss is shifted by 1

            # First round of predictions (using teacher forcing)
            optimizer.zero_grad()
            output = model(src_tensor, tgt_input, feature_mask_src, tgt_is_causal=True)

            """ # Apparently teacher forcing all the way is better now
            # Sample from model output and true target based on current sampling rate
            sampled_tgt_input = sampler.sample(output, tgt_expected)

            # Ensure proper alignment for next round of input:
            # Take the first element of each item in the target sequence (start token)
            # Concatenate with the sampled output (excluding the last token)
            sampled_tgt_input = torch.cat(
                [tgt_input[:, :1], sampled_tgt_input[:, :-1]], dim=1
            )

            # Second round of predictions using the sampled input
            output = model(src_tensor, sampled_tgt_input, feature_mask_src, tgt_is_causal=True)
            """

            # Compute the loss
            loss = criterion(output.reshape(-1, vocab_size), tgt_expected.reshape(-1))

            # Apply padding mask to loss (ignores padded tokens)
            tgt_mask = (tgt_input != pad_token).float().view(-1)
            loss = loss * tgt_mask  # Element-wise multiply with the mask to ignore padding tokens
            loss = loss.sum() / tgt_mask.sum()  # Normalize the loss (average over non-padding tokens)
            
            # Apply loss
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)  # Clip gradients
            optimizer.step()
            scheduler.step()
            # sampler.step()  # Update the sampler (teacher forcing rate)

            total_loss += loss.item()

        print(f"Epoch {epoch+1}, Loss: {total_loss * batch_size / len(train_examples)}")
        
        # Evaluate on the test set
        model.eval()
        test_loss = 0
        with torch.no_grad():
            for i in range(0, len(test_examples), batch_size):
                batch = test_examples[i:i+batch_size]
                src_batch, tgt_batch = zip(*batch)

                max_batch_len = max(max(len(s) for s in src_batch), max(len(t) for t in tgt_batch))

                # Pad sequences to the maximum length (max_len) in the batch
                src_padded = [pad_sequence(seq, max_batch_len) for seq in src_batch]
                tgt_padded = [pad_sequence(seq, max_batch_len) for seq in tgt_batch]

                # Convert padded sequences to tensors
                src_tensor = torch.tensor([tokenize(seq, char_to_idx) for seq in src_padded], device=device)
                tgt_tensor = torch.tensor([tokenize(seq, char_to_idx) for seq in tgt_padded], device=device)

                # Create the feature mask
                feature_mask_src = torch.stack([create_feature_mask(seq) for seq in src_padded], dim=0)

                # Shift target tensor
                tgt_input = tgt_tensor[:, :-1]
                tgt_expected = tgt_tensor[:, 1:]

                # Forward pass
                output = model(src_tensor, tgt_input, feature_mask_src, tgt_is_causal=True)

                # Compute the loss
                loss = criterion(output.reshape(-1, vocab_size), tgt_expected.reshape(-1))

                # Apply padding mask to loss
                tgt_mask = (tgt_expected != pad_token).float().view(-1)
                loss = loss * tgt_mask  # Element-wise multiply with the mask to ignore padding tokens
                loss = loss.sum() / tgt_mask.sum()  # Normalize the loss (average over non-padding tokens)
                
                test_loss += loss.item()

        test_loss = test_loss * batch_size / len(test_examples)
        print(f"Test Loss after Epoch {epoch+1}: {test_loss}")

        # Early stopping based on test set loss
        if test_loss < best_test_loss:
            best_test_loss = test_loss
            best_epoch = epoch
            patience_count = 0
            best_model_state = copy.deepcopy(model.state_dict())  # Save best model state
        else:
            patience_count += 1
            if patience_count == patience:
                patience_count = 0
                patience = int(patience * math.sqrt(2))
                # Rollback to best model state (undo last epoch update)
                model.load_state_dict(best_model_state)
                print(f"Rolling back to best model from epoch {best_epoch + 1}")
                print(f"Best test loss: {best_test_loss}")
        
    model.load_state_dict(best_model_state)


{' ', 'E', '<PST.PTCP>', 'ë', '<PRS.PTCP>', 'ê', 'ö', 'ï', 'w', 'R', 'o', 'q', 'i', 'h', 'T', 't', 'k', "'", 'p', 'l', '’', 'v', 's', '*', 'è', '<3SG>', '0', 'n', 'u', 'e', 'b', 'U', '4', 'æ', 'f', 'ä', '9', 'r', '6', '<PST>', 'é', '8', '<PRS>', 'z', '-', 'û', 'j', 'x', '1', 'y', 'c', 'd', 'm', 'œ', 'g', '/', 'a'}


In [7]:
for train_size in range(2500, 2501, 100):
    train_set, test_set, val_set = prepare_data(data, val_size=1000, train_size=train_size, test_ratio=0.1)
    train_examples = [ex for paradigm in train_set for ex in generate_examples(paradigm)]
    test_examples = [ex for paradigm in test_set for ex in generate_examples(paradigm)]
    print(len(train_set), len(train_examples))
    model = CharTransformer(vocab_size, device=device, max_len=max_len)
    train_model(model, train_examples, test_examples, batch_size=398, epochs=1500000//train_size)

    save_dir = "/home/minhk/Assignments/CSCI 5801/project/model/"
    filename=f"base_transformers_alt_{train_size}.pth"
    os.makedirs(save_dir, exist_ok=True)
    torch.save(model, save_dir+filename)

2250 11250


/home/minhk/Assignments/CSCI 5801/project/venv/lib/python3.10/site-packages/torch/nn/functional.py:5962: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(


Epoch 1, Loss: 3.5518573827107747
Test Loss after Epoch 1: 3.6221088119506835


/home/minhk/Assignments/CSCI 5801/project/venv/lib/python3.10/site-packages/torch/nn/modules/transformer.py:508: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. We recommend specifying layout=torch.jagged when constructing a nested tensor, as this layout receives active development, has better operator coverage, and works with torch.compile. (Triggered internally at /pytorch/aten/src/ATen/NestedTensorImpl.cpp:178.)
  output = torch._nested_tensor_from_mask(


Epoch 2, Loss: 2.858058196767171
Test Loss after Epoch 2: 2.8798387512207033
Epoch 3, Loss: 2.6143098167419434
Test Loss after Epoch 3: 2.627322847175598
Epoch 4, Loss: 2.3491372575971816
Test Loss after Epoch 4: 2.4874822744369505
Epoch 5, Loss: 2.1714695446650185
Test Loss after Epoch 5: 2.444875334739685
Epoch 6, Loss: 2.057154402669271
Test Loss after Epoch 6: 2.354931348991394
Epoch 7, Loss: 1.96064908235338
Test Loss after Epoch 7: 2.23118809261322
Epoch 8, Loss: 1.8575548332850138
Test Loss after Epoch 8: 2.1422642566680907
Epoch 9, Loss: 1.7842606801986693
Test Loss after Epoch 9: 2.078327885055542
Epoch 10, Loss: 1.724966934246487
Test Loss after Epoch 10: 2.0150924133300783
Epoch 11, Loss: 1.6728838064829508
Test Loss after Epoch 11: 1.9940813583374024
Epoch 12, Loss: 1.6380140181223553
Test Loss after Epoch 12: 1.9518109386444091
Epoch 13, Loss: 1.6004921262105305
Test Loss after Epoch 13: 1.910543930053711
Epoch 14, Loss: 1.5613300988727146
Test Loss after Epoch 14: 1.86298

In [8]:
# Select a random paradigm from validation set
random_paradigm = random.choice(val_set)

# Generate examples from the paradigm
generated_examples = generate_examples(random_paradigm)

def list_to_word(arr):
    return ''.join(arr[1:-1])

for src, tgt in generated_examples: 
    print(list_to_word(src), list_to_word(tgt))
    feature_mask = create_feature_mask(src).unsqueeze(0)
    src_tokenized = torch.tensor(tokenize(src, char_to_idx), device=device).unsqueeze(0)
    gen = model.generate(src_tokenized, feature_mask, beam_size=5).squeeze(0)
    gen = list_to_word([idx_to_char[id.item()] for id in gen])
    print(gen)


<PRS>wuff wuff
wuff
<3SG>wuff wuffs
wuffs
<PST>wuff wuffed
wufffufuffied
<PRS.PTCP>wuff wuffing
wuffing
<PST.PTCP>wuff wuffed
wufffufuffied


In [9]:
word = 'sing'
tags = ["PRS", "3SG", "PST", "PRS.PTCP", "PST.PTCP"]

for tag in tags: 
    tokens = ["<s>", f"<{tag}>"] + list(word) + ["</s>"]
    feature_mask = create_feature_mask(tokens).unsqueeze(0)
    src_tokenized = torch.tensor(tokenize(tokens, char_to_idx), device=device).unsqueeze(0)
    gen = model.generate(src_tokenized, feature_mask).squeeze(0)
    gen = list_to_word([idx_to_char[id.item()] for id in gen])
    gen2 = model.greedy_batch_decode(src_tokenized, feature_mask).squeeze(0)
    gen2 = list_to_word([idx_to_char[id.item()] for id in gen2])
    print(f"{word} + {tag} = {gen}, {gen2}")
    

sing + PRS = sing, sing
sing + 3SG = sings, sings
sing + PST = singgung, singgung
sing + PRS.PTCP = singing, singing
sing + PST.PTCP = singgung, singgung


In [10]:
# Perform greedy batched decoding
train_set, test_set, val_set = prepare_data(data, val_size=5000, train_size=train_size, test_ratio=0.1)

for train_size in range(2500, 2501, 100):
    save_dir = f"/home/minhk/Assignments/CSCI 5801/project/model/base_transformers_alt_{train_size}.pth"

    # Load model
    model = torch.load(save_dir, map_location=device, weights_only=False)
    model.to(device)
    model.eval()  # Set model to evaluation mode

    print(f"Model {train_size} loaded")

    correct_predictions = 0
    total_predictions = 0
    batch_size = 32

    all_src = []
    all_tgt = []

    for paradigm in val_set:
        pairs = generate_examples(paradigm)
        for src, tgt in pairs:
            all_src.append(src)
            all_tgt.append(tgt)

    # Determine max sequence length in the batch for padding
    def pad_sequences(sequences, pad_token, max_len=None):
        """Pads sequences to the max length with the given pad token."""
        max_len = max(len(seq) for seq in sequences)
        return [seq + [pad_token] * (max_len - len(seq)) for seq in sequences]

    # Process in batches
    for i in range(0, len(all_src), batch_size):
        src_batch = all_src[i:i+batch_size]
        tgt_batch = all_tgt[i:i+batch_size]

        try:
            # Pad inputs
            src_batch_padded = pad_sequences(src_batch, "<pad>")
            tgt_batch_padded = pad_sequences(tgt_batch, "<pad>")  # Only for consistent tensor creation
            
            feature_mask = torch.stack([create_feature_mask(src) for src in src_batch_padded]).to(device)
            src_tokenized = torch.tensor([tokenize(seq, char_to_idx) for seq in src_batch_padded], device=device)
            
            gen_batch = model.greedy_batch_decode(src_tokenized, feature_mask)
            
            for gen, tgt in zip(gen_batch, tgt_batch):
                gen_str = "".join([idx_to_char[idx.item()] for idx in gen])
                
                # Trim at first stop token
                gen_str = gen_str.split("</s>")[0][3:]
                
                correct_predictions += (gen_str == list_to_word(tgt))
                total_predictions += 1
        except:
            pass

    # Print accuracy
    print(f"Model trained on {train_size} paradigms; {correct_predictions} predictions correct out of {total_predictions} total. Accuracy: {correct_predictions / total_predictions:.4f}")


Model 2500 loaded
Model trained on 2500 paradigms; 23612 predictions correct out of 25000 total. Accuracy: 0.9445


In [11]:
correct_predictions = 0
total_predictions = 0

for paradigm in val_set:
    pairs = generate_examples(paradigm)
    for src, tgt in pairs: 
        feature_mask = create_feature_mask(src).unsqueeze(0)
        src_tokenized = torch.tensor(tokenize(src, char_to_idx), device=device).unsqueeze(0)
        gen = model.generate(src_tokenized, feature_mask).squeeze(0)
        gen = [idx_to_char[id.item()] for id in gen]
        correct_predictions += (gen == tgt)
        total_predictions += 1

print(f"{correct_predictions} predictions correct out of {total_predictions} total. Accuracy: {correct_predictions / total_predictions}")
    

20189 predictions correct out of 25000 total. Accuracy: 0.80756
